# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import requests # execute api calls to iex 
import numpy as np 
import pandas as pd
import xlsxwriter 
import math 


stocks = pd.read_csv("/Users/kp/algorithmic-trading-python/starter_files/sp_500_stocks.csv")
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [4]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [6]:
symbol = "AAPL"
url  = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
response = requests.get(url).json()
response

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'TT  NKSAE A)RD/MEB(GQLNOCALSLEGAS',
 'calculationPrice': 'close',
 'open': 139.7,
 'openTime': 1685614864606,
 'openSource': 'ioclfaif',
 'close': 135.22,
 'closeTime': 1679636044231,
 'closeSource': 'oiffaicl',
 'high': 137.25,
 'highTime': 1624614835599,
 'highSource': 'pi5 lieady dcu 1mrneeet',
 'low': 140.21,
 'lowTime': 1671330172854,
 'lowSource': 'd ei irdemnyeea clutp51',
 'latestPrice': 135.14,
 'latestSource': 'Close',
 'latestTime': 'February 2, 2021',
 'latestUpdate': 1681019991133,
 'latestVolume': 83700399,
 'iexRealtimePrice': 140.66,
 'iexRealtimeSize': 4,
 'iexLastUpdated': 1613224056679,
 'delayedPrice': 139.46,
 'delayedPriceTime': 1612908949127,
 'oddLotDelayedPrice': 140.65,
 'oddLotDelayedPriceTime': 1650575096074,
 'extendedPrice': 142,
 'extendedChange': 0.37,
 'extendedChangePercent': 0.00275,
 'extendedPriceTime': 1613465494952,
 'previousClose': 138.92,
 'previousVolume': 109663537,
 'change

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [7]:
price = response['latestPrice']
market_cap = response['marketCap']
market_cap

2305529243291

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [8]:
columns  = ['Ticker','Stock Price','Market Capitalization','Number of Shares to buy']
data = pd.DataFrame(columns=columns)
data = data.append(pd.Series([symbol,price,market_cap,np.nan],index = columns),ignore_index = True)
data

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,AAPL,135.14,2305529243291,NaN


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [9]:
data = pd.DataFrame(columns = columns)
for stock in stocks['Ticker']:
    url  = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    response = requests.get(url).json()
    data = data.append(pd.Series([stock,response['latestPrice'],response['marketCap'],np.nan],index = columns),ignore_index = True)



KeyboardInterrupt: 

In [ ]:
data

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

data = pd.DataFrame(columns = columns)
for symbol_string in symbol_strings:
    batch_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    response = requests.get(batch_url).json()
    for symbol in symbol_string.split(","):
        data = data.append(pd.Series([symbol,response[symbol]['quote']['latestPrice'],
                                      response[symbol]['quote']['marketCap'],np.nan],index = columns),ignore_index = True)
        
        
        


In [13]:
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [14]:
portfolio_size = input("Enter the value of your portfolio")

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \n Please try again:")
    portfolio_size = input("Enter the value of your portfolio")
    val = float(portfolio_size)

Enter the value of your portfolio10000000
10000000.0


In [47]:
position_size = val/len(data.index)

data['Number of Shares to buy']  = data.apply(lambda x:position_size/x['Stock Price'],axis = 1)
#data.set_index('Ticker',inplace = True)
data

,Stock Price,Market Capitalization,Number of Shares to buy
Ticker,,,
A,123.41,39612927411,160.456853
AAL,17.36,10256744838,1140.667062
AAP,160.19,10848668295,123.615583
AAPL,141.01,2365137981364,140.429616
ABBV,105.84,186027574440,187.093539
ABC,110.72,22590716755,178.847365
ABMD,361.75,16227630375,54.739406
ABT,126.18,216914432712,156.934381
ACN,259.35,166692697601,76.352343


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [51]:
writer = pd.ExcelWriter('Recommended_trades.xlsx',engine = 'xlsxwriter')
data.to_excel(writer,'Recommended_Trades')

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [52]:
background_colour = '#0a0a23'
font_colour = '#ffffff'
string_format = writer.book.add_format(
    {
        'font_color': font_colour,
        'bg_color': background_colour,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color': font_colour,
        'bg_color': background_colour,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_colour,
        'bg_color': background_colour,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [53]:
column_formats  = {
    'A':['Ticker',string_format],
    'B':['Stock Price',dollar_format],
    'C':['Market Capitalization',dollar_format],
    'D':['Number of Shares to buy',integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended_Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
    writer.sheets['Recommended_Trades'].write(f'{column}1',column_formats[column][0],column_formats[column][1])
writer.save()

## Saving Our Excel Output

Saving our Excel file is very easy: